In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [2]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


## Wczytywanie danych

In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [6]:
cat_feats = [ x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [ x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, cat_feats):
  X = df[cat_feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTreeRegressor

In [8]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)


(-19695.13091100928, 148.72570644015937)

## RandomForest

In [9]:
model=RandomForestRegressor(max_depth=5, n_estimators = 50, random_state=0)
run_model(model, cat_feats)

(-18718.657185256638, 64.5424578125788)

## XGBoost

In [10]:
xgb_params = {
    'max_depth': 5,
    'n_estimators' : 50,
    'learning_rate' :0.1,
    'seed' : 0
 }

run_model(xgb.XGBRegressor(**xgb_params),cat_feats)

[21:25:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:25:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:26:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [13]:
X = df[cat_feats].values
y = df['price_value'].values

m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[21:27:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [14]:
feats = [
  'param_napęd__cat',
	'param_rok-produkcji__cat',
	'param_stan__cat',
	'param_skrzynia-biegów__cat',
	'param_faktura-vat__cat',
	'param_moc__cat',
	'param_marka-pojazdu__cat',
	'feature_kamera-cofania__cat',
	'param_typ__cat',
	'param_pojemność-skokowa__cat',
	'seller_name__cat',
	'feature_wspomaganie-kierownicy__cat',
	'param_model-pojazdu__cat',
	'param_wersja__cat',
	'param_kod-silnika__cat',
	'feature_system-start-stop__cat',
	'feature_asystent-pasa-ruchu__cat',
	'feature_czujniki-parkowania-przednie__cat',
	'feature_łopatki-zmiany-biegów__cat',
	'feature_regulowane-zawieszenie__cat']
  
run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:33:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [15]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = [
  'param_napęd__cat',
	'param_rok-produkcji',
	'param_stan__cat',
	'param_skrzynia-biegów__cat',
	'param_faktura-vat__cat',
	'param_moc__cat',
	'param_marka-pojazdu__cat',
	'feature_kamera-cofania__cat',
	'param_typ__cat',
	'param_pojemność-skokowa__cat',
	'seller_name__cat',
	'feature_wspomaganie-kierownicy__cat',
	'param_model-pojazdu__cat',
	'param_wersja__cat',
	'param_kod-silnika__cat',
	'feature_system-start-stop__cat',
	'feature_asystent-pasa-ruchu__cat',
	'feature_czujniki-parkowania-przednie__cat',
	'feature_łopatki-zmiany-biegów__cat',
	'feature_regulowane-zawieszenie__cat']
  
run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:33:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [16]:
df['param_moc'] = df['param_moc'].map(lambda x:  -1 if str(x) == 'None' else int(str(x).split(' ')[0]))

df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))

feats = [
  'param_napęd__cat',
	'param_rok-produkcji',
	'param_stan__cat',
	'param_skrzynia-biegów__cat',
	'param_faktura-vat__cat',
	'param_moc',
	'param_marka-pojazdu__cat',
	'feature_kamera-cofania__cat',
	'param_typ__cat',
	'param_pojemność-skokowa__cat',
	'seller_name__cat',
	'feature_wspomaganie-kierownicy__cat',
	'param_model-pojazdu__cat',
	'param_wersja__cat',
	'param_kod-silnika__cat',
	'feature_system-start-stop__cat',
	'feature_asystent-pasa-ruchu__cat',
	'feature_czujniki-parkowania-przednie__cat',
	'feature_łopatki-zmiany-biegów__cat',
	'feature_regulowane-zawieszenie__cat']
  
run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:33:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:33:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.450230340148, 62.2164408321879)

In [17]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x:  -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')))

feats = [
  'param_napęd__cat',
	'param_rok-produkcji',
	'param_stan__cat',
	'param_skrzynia-biegów__cat',
	'param_faktura-vat__cat',
	'param_moc',
	'param_marka-pojazdu__cat',
	'feature_kamera-cofania__cat',
	'param_typ__cat',
	'param_pojemność-skokowa',
	'seller_name__cat',
	'feature_wspomaganie-kierownicy__cat',
	'param_model-pojazdu__cat',
	'param_wersja__cat',
	'param_kod-silnika__cat',
	'feature_system-start-stop__cat',
	'feature_asystent-pasa-ruchu__cat',
	'feature_czujniki-parkowania-przednie__cat',
	'feature_łopatki-zmiany-biegów__cat',
	'feature_regulowane-zawieszenie__cat']
  
run_model(xgb.XGBRegressor(**xgb_params), feats)


[21:34:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:34:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.227198767323, 72.83561801421891)